# Рекомендательная система
В данном проекте мы рассмотрим основные этапы создания рекомендательной системы интернет-магазина.

Основной целью проекта является изучение принципов работы рекомендательных систем, знакомство с различными алгоритмами и создание прототипа.

В ходе работы над проектом были решены следующие задачи:

1. EDA - разведывательный анализ данных

2. Предобработка данных и генерация новых признаков

3. Обучение простой модели LightFM

In [121]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/recommendationsv4/sample_submission.csv
/kaggle/input/recommendationsv4/train.csv
/kaggle/input/recommendationsv4/test.csv
/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json


In [122]:
#Читаем файлы
train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv')
test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv')
submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [123]:
import numpy as np
import pandas as pd
from collections import Counter
import json
import re
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

import scipy.sparse as sparse

pd.set_option('display.max_columns', None)

In [124]:

# Заменяем True/False на 1 и 0
true_false = {True: 1, False: 0}

In [125]:
def check_df_before_merg(d_df1,d_df2,n_line):
    
    list_of_names1 = list(d_df1.columns)
    temp_dict = {}
    temp_dict['# уник_1'] = d_df1.nunique().values
    temp_dict[f'в  строке_1 ({n_line} позиц-я)'] =d_df1.loc[n_line].values
    temp_dict['тип_1'] = d_df1.dtypes
    temp_dict['имя признака_1'] = list_of_names1
    temp_df1 = pd.DataFrame.from_dict(temp_dict)
    
    
    list_of_names2 = list(d_df2.columns)
    temp_dict2 = {}
    temp_dict2['имя признака_2'] = list_of_names2
    temp_dict2['тип_2'] = d_df2.dtypes
    temp_dict2[f'в  строке_2 ({n_line} позиц-я)'] =d_df2.loc[n_line].values
    temp_dict2['# уник_2'] = d_df2.nunique().values
    temp_df2 = pd.DataFrame.from_dict(temp_dict2)
    
    temp_df = pd.concat([temp_df1,temp_df2], axis=1, sort=False)
    temp_df.reset_index(inplace = True)
    del temp_df['index']
    display(temp_df)

    temp_dict3 = {}
    temp_df3= pd.DataFrame(temp_df)
    temp_list  = []
    temp_list2  = []
    temp_list3  = []
    temp_list4  = []
    temp_list5  = []

    for i in range(len(temp_df)):
        if str(temp_df3['тип_2'][i]) != str(temp_df3['тип_1'][i]):
            temp_list.append(temp_df3['имя признака_1'][i])
            temp_list2.append(temp_df3['имя признака_2'][i])
            temp_list3.append(str(temp_df3['тип_1'][i]) + '!=' + str(temp_df3['тип_2'][i]))
            temp_list4.append(i)
        if temp_df3['# уник_2'][i]>0 and temp_df3['# уник_1'][i]/temp_df3['# уник_2'][i] > 2:
            temp_list5.append(i)
            
    temp_dict3['index']= temp_list4
    temp_dict3['имя признака_1']= temp_list
    temp_dict3['не совпадают типы'] = temp_list3
    temp_dict3['имя признака_2']= temp_list2

    temp_df4 = pd.DataFrame.from_dict(temp_dict3)
    temp_df4.set_index('index',inplace=True)

    print(f'Резюме:\n 1. Не совпали типы в:= {len(temp_df4)} столбцах\n')
    print(f'2. Уникальные значения различаются в:= {len(temp_list5)} столбцах {temp_list5}')
    display(temp_df4)



    return

def uniq_nan (df,i):
# Уникальные значения, пропущенные значения 
    column_type_numeric = False
    # тип колонки
    if df[col_names[i]].dtype == 'O':
        print("Столбец:", col_names[i], "не является числовым")
    elif df[col_names[i]].dtype == "int64" or df[col_names[i]].dtype == "float64":
        column_type_numeric = True        
        print("Столбец:", col_names[i], "является числовым")
    else:
        print('Признак', col_names[i], 'имеет тип', df[col_names[i]].dtype)
    #print("Столбец:", c)    
    print(pd.DataFrame(df[col_names[i]].unique()))    
    print("Пропущено значений:", df[col_names[i]].isna().sum())
    print(df[col_names[i]].value_counts())
    
# Функция без гистограммы
def col_info(ys):
    print('Количество пропусков: {},'.format(ys.isnull().sum()))
    print('{},'.format(ys.describe()))
    print('Распределение:\n{},'.format(ys.value_counts()))

# Функция с гистограммой
def col_info_hist(ys):
    print('Количество пропусков: {},'.format(ys.isnull().sum()))
    print('{},'.format(ys.describe()))
    print('Распределение:\n{},'.format(ys.value_counts()))
    ys.hist()
    
def viz_counter_bar(df, column, title, max_values=0, sort_by_index=False):
    """
    Визуализация количества значений в ДФ, Value_counts, 
    горизонтальные столбцы
    df - датафрейм, column - столбец, title - подпись
    max_values - максимальное количество значений для отображения, 0 - все
    sort_by_index - сортирует по индексам, а не значениям
    dollars - отображает подписи с $$ для price_range
    """
    if max_values > 0:
        col_values = df[column].value_counts().nlargest(
            max_values).sort_values(ascending=True)
    else:
        col_values = df[column].value_counts(ascending=True)
    
    if sort_by_index: 
        col_values = col_values.sort_index()
    
    fig, ax = plt.subplots()
    plt.rcParams['figure.figsize'] = (8, 8)
    plt.figure 
    ax = col_values.plot(kind='bar', title=title)
    for i, v in enumerate(col_values):
        plt.text(i, v+(col_values.max()/100), ""+str(v), ha='center', rotation = 'horizontal')
    
    plt.xticks(rotation=0)
    
    plt.show()

In [126]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/recommendationsv4/sample_submission.csv
/kaggle/input/recommendationsv4/train.csv
/kaggle/input/recommendationsv4/test.csv
/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json


In [127]:
with open ('/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json') as f:
    meta=[]
    for line in f.readlines():
        meta.append(json.loads(line))
meta = pd.DataFrame(meta)

# удаляем дубликаты из json
meta = meta.iloc[meta.astype(str).drop_duplicates().index]


In [128]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283512 entries, 0 to 287208
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   category      283512 non-null  object
 1   description   248546 non-null  object
 2   title         283508 non-null  object
 3   brand         272270 non-null  object
 4   rank          271885 non-null  object
 5   also_view     118924 non-null  object
 6   main_cat      282019 non-null  object
 7   price         130234 non-null  object
 8   asin          283512 non-null  object
 9   also_buy      82308 non-null   object
 10  image         149027 non-null  object
 11  date          9575 non-null    object
 12  feature       16218 non-null   object
 13  details       269753 non-null  object
 14  similar_item  248 non-null     object
 15  tech1         743 non-null     object
 16  fit           4 non-null       object
dtypes: object(17)
memory usage: 38.9+ MB


In [129]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857895 entries, 0 to 857894
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         857895 non-null  float64
 1   verified        857895 non-null  bool   
 2   reviewTime      857895 non-null  object 
 3   asin            857895 non-null  object 
 4   reviewerName    857719 non-null  object 
 5   reviewText      857616 non-null  object 
 6   summary         857725 non-null  object 
 7   unixReviewTime  857895 non-null  int64  
 8   vote            118740 non-null  object 
 9   style           444406 non-null  object 
 10  image           7142 non-null    object 
 11  userid          857895 non-null  int64  
 12  itemid          857895 non-null  int64  
 13  rating          857895 non-null  float64
dtypes: bool(1), float64(2), int64(3), object(8)
memory usage: 85.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285965 entries, 0 to 285964
Dat

test.csv - набор данных, для которого вы должны сделать предсказания. У каждого наобора userid, itemid есть свой id, для которого вы должны сделать предсказание.

overall - рейтинг, который поставил пользователь
verified - был ли отзыв верифицирован
reviewTime - когда был отзыв написан
reviewerName - имя пользователя
reviewText - текст отзыва
summary - сжатый отзыв
vote - количество голосований за отзыв
style - метаданные
image - изображение продукта
userid - id пользователя
itemid - id товара
id - id для предсказания

In [130]:
check_df_before_merg(test,train,3)

,# уник_1,в строке_1 (3 позиц-я),тип_1,имя признака_1,имя признака_2,тип_2,в строке_2 (3 позиц-я),# уник_2
0,2.0,True,bool,verified,verified,bool,True,2.0
1,4349.0,"10 5, 2014",object,reviewTime,reviewTime,object,"09 5, 2013",4580.0
2,37876.0,B003SO8HGE,object,asin,asin,object,B00MRAQUM8,41302.0
3,86815.0,Vanessa,object,reviewerName,reviewerName,object,Connie L. Reaves,101207.0
4,4349.0,1412467200,int64,unixReviewTime,unixReviewTime,int64,1378339200,4580.0
5,304.0,NaN,object,vote,vote,object,NaN,477.0
6,18904.0,{'Flavor:': ' Sweet Rose'},object,style,style,object,NaN,25892.0
7,2306.0,NaN,object,image,image,object,NaN,6636.0
8,109357.0,26206,int64,userid,userid,int64,72854,127448.0
9,37876.0,11957,int64,itemid,itemid,int64,30249,41302.0


Резюме:
 1. Не совпали типы в:= 5 столбцах

2. Уникальные значения различаются в:= 0 столбцах []


,имя признака_1,не совпадают типы,имя признака_2
index,,,
10,Id,int64!=nan,NaN
11,NaN,nan!=float64,overall
12,NaN,nan!=object,reviewText
13,NaN,nan!=object,summary
14,NaN,nan!=float64,rating


В тренировочном и тестовом датасетах не совпадают колонки. В тестовом датасеье отсуттвуют колонки overall, reviewText, summary, rating. В тренировочном датасете нет колонки ID

# DATA PREPROCESSING

In [131]:
# Помечаем и объеденяем тренировочный и тестовый датафрейм
train['train'] = 1
test['train'] = 0

df = pd.concat([train, test])

Объединим тренировочный датасет и данные из meta по идентификатору asin (Amazon Standard Identification Number)

In [132]:
df_all = df.merge(meta,how='left', on = 'asin')

In [133]:
# максимально чистим память , на счету каждый свободный гиг
del train
del test
del meta

In [134]:
df = df_all.copy()

In [135]:
# Меняем ссылки на картинки просто на значение есть или нет
df.loc[df.image_x.isna(), 'image_x'] = 'no'
df.loc[df.image_x!='no', 'image_x'] = 'yes'

df.loc[df.image_y.isna(), 'image_y'] = 'no'
df.loc[df.image_y!='no', 'image_y'] = 'yes'

In [136]:
df.sample(2)

,overall,verified,reviewTime,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image_x,userid,itemid,rating,train,Id,category,description,title,brand,rank,also_view,main_cat,price,also_buy,image_y,date,feature,details,similar_item,tech1,fit
872701,NaN,True,"05 30, 2014",B01H0XDZPG,Danny Murphy,NaN,NaN,1401408000,NaN,NaN,no,22215,38205,NaN,0,14806.0,"[Grocery & Gourmet Food, Condiments & Salad Dr...",NaN,"Skillet Bacon Spread, 13 Oz",Skillet,"573,712inGroceryGourmetFood(",NaN,Grocery,NaN,"[B00I5OMFWO, B00LMGPDQ2, B0743JNHCF, B00T6PBBKS]",no,NaN,NaN,"\n\n\n\n\n\n <div class=""disclaim"">Size: <s...",NaN,NaN,NaN
816889,1.0,True,"02 7, 2016",B000K31LT6,N. Gabriel,If you are a true connoisseur of green and jas...,"Save yourself, and your tummy. Try a different...",1454803200,3.0,"{'Size:': ' 100', 'Flavor:': ' Jasmine'}",no,53673,3027,0.0,1,NaN,"[Grocery & Gourmet Food, Beverages, Coffee, Te...",[Prince of Peace Organic Green Tea Jasmine Des...,"Prince of Peace Organic Green Tea Jasmine, 100...",Prince Of Peace,"65,910inHealthHousehold(","[B00ZGXUPX8, B002LMD8P4, B0010BQB6A, B002LM3RI...",Health & Personal Care,NaN,"[B002LMD8P4, B0010BQB6A, B00ZGXUPX8, B00AP70BK...",yes,NaN,NaN,"\n\n\n\n\n\n <div class=""disclaim"">Flavor: ...",NaN,NaN,NaN


# EDA

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1143860 entries, 0 to 1143859
Data columns (total 32 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   overall         857895 non-null   float64
 1   verified        1143860 non-null  bool   
 2   reviewTime      1143860 non-null  object 
 3   asin            1143860 non-null  object 
 4   reviewerName    1143637 non-null  object 
 5   reviewText      857616 non-null   object 
 6   summary         857725 non-null   object 
 7   unixReviewTime  1143860 non-null  int64  
 8   vote            158202 non-null   object 
 9   style           592086 non-null   object 
 10  image_x         1143860 non-null  object 
 11  userid          1143860 non-null  int64  
 12  itemid          1143860 non-null  int64  
 13  rating          857895 non-null   float64
 14  train           1143860 non-null  int64  
 15  Id              285965 non-null   float64
 16  category        1143860 non-null  ob

Проверим пропущенные данные

In [138]:
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

overall - 25%
verified - 0%
reviewTime - 0%
asin - 0%
reviewerName - 0%
reviewText - 25%
summary - 25%
unixReviewTime - 0%
vote - 86%
style - 48%
image_x - 0%
userid - 0%
itemid - 0%
rating - 25%
train - 0%
Id - 75%
category - 0%
description - 8%
title - 0%
brand - 1%
rank - 4%
also_view - 45%
main_cat - 0%
price - 30%
also_buy - 14%
image_y - 0%
date - 99%
feature - 83%
details - 2%
similar_item - 100%
tech1 - 99%
fit - 100%


Видно, что части столбцов количество пропущенных данных составляет более 60%. Для работы данные колонки будут удалены

In [139]:
len(df)

1143860

In [140]:
cols_to_delete = df.columns[df.isna().sum() > len(df)*0.6]
df.drop(cols_to_delete, axis = 1, inplace = True)


Проверим удалились ли ненужные колонки и какие остались

In [141]:
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

overall - 25%
verified - 0%
reviewTime - 0%
asin - 0%
reviewerName - 0%
reviewText - 25%
summary - 25%
unixReviewTime - 0%
style - 48%
image_x - 0%
userid - 0%
itemid - 0%
rating - 25%
train - 0%
category - 0%
description - 8%
title - 0%
brand - 1%
rank - 4%
also_view - 45%
main_cat - 0%
price - 30%
also_buy - 14%
image_y - 0%
details - 2%


## Рассмотрим данные в столбцах

In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1143860 entries, 0 to 1143859
Data columns (total 25 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   overall         857895 non-null   float64
 1   verified        1143860 non-null  bool   
 2   reviewTime      1143860 non-null  object 
 3   asin            1143860 non-null  object 
 4   reviewerName    1143637 non-null  object 
 5   reviewText      857616 non-null   object 
 6   summary         857725 non-null   object 
 7   unixReviewTime  1143860 non-null  int64  
 8   style           592086 non-null   object 
 9   image_x         1143860 non-null  object 
 10  userid          1143860 non-null  int64  
 11  itemid          1143860 non-null  int64  
 12  rating          857895 non-null   float64
 13  train           1143860 non-null  int64  
 14  category        1143860 non-null  object 
 15  description     1047984 non-null  object 
 16  title           1143860 non-null  ob

Числовые признаки
overall - оценка пользователей
price - цена товара

Бинарные признаки
verified - подтвержденный аккаунт
image_x - есть иконка или нет
image_y - есть фотографии изображений или нет (предпросмотр)
rating - понравился товар или нет (1 - да, 0 - нет)

Временные ряды
reviewtime - дата отзыва
unixreviewtime - время отзыва в формате unix

Категориальные признаки ( в т.ч. текст)
item_id - id товара (аналог asin)
reviewerName - имя/ник пользователя
reviewtext - полный текст отзыва покупателя
summary - краткий(итоговый) отзыв от покупателя
style - вид упаковки
category - категория товаров
description - описание товара от производителя
title - полное название (включая производителя и особенность)
brand - Бренд товара
rank - ранк товара
main_cat - основная категория

train, id, userid, asin - идентификаторы и рассмотрены не будут

В тестовом датасеье отсуттвуют колонки overall, reviewText, summary, rating. В тренировочном датасете нет колонки ID

In [143]:
df.columns

Index(['overall', 'verified', 'reviewTime', 'asin', 'reviewerName',
       'reviewText', 'summary', 'unixReviewTime', 'style', 'image_x', 'userid',
       'itemid', 'rating', 'train', 'category', 'description', 'title',
       'brand', 'rank', 'also_view', 'main_cat', 'price', 'also_buy',
       'image_y', 'details'],
      dtype='object')

### overall

In [144]:
col_info(df.overall)

Количество пропусков: 285965,
count    857895.000000
mean          4.441695
std           1.064655
min           1.000000
25%           4.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: overall, dtype: float64,
Распределение:
5.0    615196
4.0    113094
3.0     60440
1.0     37498
2.0     31667
Name: overall, dtype: int64,


Не требует вмешательства

In [145]:
df[['overall', 'rating']].corr()

,overall,rating
overall,1.000000,0.897339
rating,0.897339,1.000000


### verified

In [146]:
col_info(df.verified)

Количество пропусков: 0,
count     1143860
unique          2
top          True
freq       993847
Name: verified, dtype: object,
Распределение:
True     993847
False    150013
Name: verified, dtype: int64,


Бинарный признак. Данный признак скорее всего отражает подтвержден пользователь или нет. Пропусков нет

In [147]:
df[['verified', 'rating']].corr()

,verified,rating
verified,1.000000,0.072735
rating,0.072735,1.000000


In [148]:
# Заменяем True/False на 1 и 0
true_false = {True: 1, False: 0}

In [149]:
df['verified'] = df['verified'].map(true_false)

### unixReviewTime, reviewTime¶

In [150]:
df['reviewTime_new'] = pd.to_datetime(df.unixReviewTime, unit='s')

In [151]:
df = df.drop(columns=['reviewTime'])

In [152]:
df = df.rename(columns={'reviewTime_new': 'reviewTime'})

In [153]:
df.query('train==1')[['unixReviewTime', 'rating']].corr()

,unixReviewTime,rating
unixReviewTime,1.000000,0.013242
rating,0.013242,1.000000


Низкая корреляция с рейтингом. Скорее всего дата отзыва не влияет на рейтинг товара. Столбцы можно удалить

### reviewerName

In [154]:
col_info(df.reviewerName)

Количество пропусков: 223,
count             1143637
unique             101473
top       Amazon Customer
freq                42368
Name: reviewerName, dtype: object,
Распределение:
Amazon Customer    42368
Kindle Customer     8288
Linda                988
John                 950
David                808
                   ...  
Amazon17               1
The Jajh               1
Jelane                 1
Crazywolf              1
RovingIrish            1
Name: reviewerName, Length: 101473, dtype: int64,


Для данного столбца есть два натболее значимых имени. Предлагается оставить два этих имени, остальные, включая пропуски отнести в категорию прочие

In [155]:
df['reviewerName'] = df.reviewerName.apply(lambda x: x if x == "Amazon Customer" or x == "Kindle Customer" else "Other Customer") 

In [156]:
col_info(df.reviewerName)

Количество пропусков: 0,
count            1143860
unique                 3
top       Other Customer
freq             1093204
Name: reviewerName, dtype: object,
Распределение:
Other Customer     1093204
Amazon Customer      42368
Kindle Customer       8288
Name: reviewerName, dtype: int64,


### reviewText

In [157]:
col_info(df.reviewText)

Количество пропусков: 286244,
count     857616
unique    686739
top         good
freq        3144
Name: reviewText, dtype: object,
Распределение:
good                                                                                                                                                                                                                                                                                                                     3144
great                                                                                                                                                                                                                                                                                                                    2612
Great                                                                                                                                                                                                                     

In [158]:
df.reviewText[51]

"Best raw honey I've ordered at this price range. I love the texture and taste. 10/10 highly recommend"

В тестовом наборе нет этих данных. На данном этапе признак удалим

In [159]:
df.drop(['reviewText'], axis=1, inplace=True)

### summary

In [160]:
col_info(df.summary)

Количество пропусков: 286135,
count         857725
unique        411451
top       Five Stars
freq          173850
Name: summary, dtype: object,
Распределение:
Five Stars                      173850
Four Stars                       26878
Three Stars                      12033
One Star                          5386
Two Stars                         4989
                                 ...  
He Loved It!                         1
Best Brand of Steel Cut Oats         1
Just yummy                           1
Skittles Sour                        1
Always good, all the time!           1
Name: summary, Length: 411451, dtype: int64,


In [161]:
df.summary[70]

'Yum'

В тестовом наборе нет этих данных. На данном этапе признак удалим

In [162]:
df.drop(['summary'], axis=1, inplace=True)

### itemid

In [163]:
col_info(df.itemid)

Количество пропусков: 0,
count    1.143860e+06
mean     1.708120e+04
std      1.143778e+04
min      0.000000e+00
25%      6.329000e+03
50%      1.593500e+04
75%      2.661600e+04
max      4.131900e+04
Name: itemid, dtype: float64,
Распределение:
22208    7387
24556    6228
23540    6221
32046    3387
5195     3030
         ... 
39004       1
38902       1
38958       1
38965       1
38996       1
Name: itemid, Length: 41320, dtype: int64,


In [164]:
df.groupby('asin')['rating'].agg(['count', 'mean']).sort_values(by='count', ascending=False).head(3)

,count,mean
asin,,
B00BUKL666,5491,0.884720
B00D3M2QP4,4733,0.886753
B008QMX2SG,4654,0.886764


In [165]:
df.groupby('itemid')['rating'].agg(['count', 'mean']).sort_values(by='count', ascending=False).head(3)

,count,mean
itemid,,
22208,5491,0.884720
23540,4733,0.886753
24556,4654,0.886764


In [166]:
#Данные совпадают. Скорее всего это одно и тоже. Один из признаков удалим

### category

In [167]:
df.category[70]

['Grocery & Gourmet Food', 'Snack Foods', 'Crackers']

In [168]:
new_df = df['category'].astype(str)

In [169]:
new_df = new_df.str.split(',', expand=True)

In [170]:
new_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
0,['Grocery & Gourmet Food','Herbs,Spices & Seasonings','Mixed Spices & Seasonings','Adobo Seasoning'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,['Grocery & Gourmet Food','Beverages','Coffee,Tea & Cocoa','Cocoa'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,['Grocery & Gourmet Food','Beverages','Bottled Beverages,Water & Drink Mixes','Flavor Syrups'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,['Grocery & Gourmet Food','Herbs,Spices & Seasonings','Single Herbs & Spices','Cinnamon'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,['Grocery & Gourmet Food','Snack Foods','Chips & Crisps','Potato'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143855,['Grocery & Gourmet Food','Cooking & Baking','Sugar Substitutes'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1143856,['Grocery & Gourmet Food','Snack Foods','Crackers','Assortments & Samplers'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1143857,['Grocery & Gourmet Food','Beverages','Coffee,Tea & Cocoa','Coffee','Roasted Coffee Beans'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1143858,['Grocery & Gourmet Food','Produce','Dried Fruits & Vegetables','Dried Fruits','Dried Dates'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [171]:
for col in new_df.columns:
    pct_missing = np.mean(new_df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

0 - 0%
1 - 0%
2 - 1%
3 - 16%
4 - 51%
5 - 74%
6 - 99%
7 - 100%
8 - 100%
9 - 100%
10 - 100%
11 - 100%
12 - 100%
13 - 100%
14 - 100%
15 - 100%
16 - 100%
17 - 100%
18 - 100%
19 - 100%
20 - 100%
21 - 100%
22 - 100%
23 - 100%
24 - 100%
25 - 100%
26 - 100%
27 - 100%
28 - 100%
29 - 100%
30 - 100%
31 - 100%
32 - 100%
33 - 100%
34 - 100%
35 - 100%
36 - 100%
37 - 100%


In [172]:
cols_to_delete = new_df.columns[new_df.isna().sum() > len(df)*0.99]
new_df.drop(cols_to_delete, axis = 1, inplace = True)

In [173]:
new_df

,0,1,2,3,4,5,6
0,['Grocery & Gourmet Food','Herbs,Spices & Seasonings','Mixed Spices & Seasonings','Adobo Seasoning'],None,None
1,['Grocery & Gourmet Food','Beverages','Coffee,Tea & Cocoa','Cocoa'],None,None
2,['Grocery & Gourmet Food','Beverages','Bottled Beverages,Water & Drink Mixes','Flavor Syrups'],None,None
3,['Grocery & Gourmet Food','Herbs,Spices & Seasonings','Single Herbs & Spices','Cinnamon'],None,None
4,['Grocery & Gourmet Food','Snack Foods','Chips & Crisps','Potato'],None,None,None
...,...,...,...,...,...,...,...
1143855,['Grocery & Gourmet Food','Cooking & Baking','Sugar Substitutes'],None,None,None,None
1143856,['Grocery & Gourmet Food','Snack Foods','Crackers','Assortments & Samplers'],None,None,None
1143857,['Grocery & Gourmet Food','Beverages','Coffee,Tea & Cocoa','Coffee','Roasted Coffee Beans'],None
1143858,['Grocery & Gourmet Food','Produce','Dried Fruits & Vegetables','Dried Fruits','Dried Dates'],None,None


In [174]:
for col in new_df.columns:
    new_df[col] = new_df[col].fillna(0)

In [175]:
new_df

,0,1,2,3,4,5,6
0,['Grocery & Gourmet Food','Herbs,Spices & Seasonings','Mixed Spices & Seasonings','Adobo Seasoning'],0,0
1,['Grocery & Gourmet Food','Beverages','Coffee,Tea & Cocoa','Cocoa'],0,0
2,['Grocery & Gourmet Food','Beverages','Bottled Beverages,Water & Drink Mixes','Flavor Syrups'],0,0
3,['Grocery & Gourmet Food','Herbs,Spices & Seasonings','Single Herbs & Spices','Cinnamon'],0,0
4,['Grocery & Gourmet Food','Snack Foods','Chips & Crisps','Potato'],0,0,0
...,...,...,...,...,...,...,...
1143855,['Grocery & Gourmet Food','Cooking & Baking','Sugar Substitutes'],0,0,0,0
1143856,['Grocery & Gourmet Food','Snack Foods','Crackers','Assortments & Samplers'],0,0,0
1143857,['Grocery & Gourmet Food','Beverages','Coffee,Tea & Cocoa','Coffee','Roasted Coffee Beans'],0
1143858,['Grocery & Gourmet Food','Produce','Dried Fruits & Vegetables','Dried Fruits','Dried Dates'],0,0


In [176]:
new_df

,0,1,2,3,4,5,6
0,['Grocery & Gourmet Food','Herbs,Spices & Seasonings','Mixed Spices & Seasonings','Adobo Seasoning'],0,0
1,['Grocery & Gourmet Food','Beverages','Coffee,Tea & Cocoa','Cocoa'],0,0
2,['Grocery & Gourmet Food','Beverages','Bottled Beverages,Water & Drink Mixes','Flavor Syrups'],0,0
3,['Grocery & Gourmet Food','Herbs,Spices & Seasonings','Single Herbs & Spices','Cinnamon'],0,0
4,['Grocery & Gourmet Food','Snack Foods','Chips & Crisps','Potato'],0,0,0
...,...,...,...,...,...,...,...
1143855,['Grocery & Gourmet Food','Cooking & Baking','Sugar Substitutes'],0,0,0,0
1143856,['Grocery & Gourmet Food','Snack Foods','Crackers','Assortments & Samplers'],0,0,0
1143857,['Grocery & Gourmet Food','Beverages','Coffee,Tea & Cocoa','Coffee','Roasted Coffee Beans'],0
1143858,['Grocery & Gourmet Food','Produce','Dried Fruits & Vegetables','Dried Fruits','Dried Dates'],0,0


In [177]:
new_df1=new_df[1]
new_df1 = new_df1.str.replace(']','')
col_info(new_df1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


Количество пропусков: 0,
count          1143860
unique              25
top        'Beverages'
freq            301419
Name: 1, dtype: object,
Распределение:
 'Beverages'                             301419
 'Cooking & Baking'                      223358
 'Snack Foods'                           142090
 'Candy & Chocolate'                     122730
 'Herbs                                   74659
 'Breakfast Foods'                        54217
 'Canned                                  41192
 'Produce'                                26631
 'Soups                                   24030
 'Sauces                                  21073
 'Jams                                    20521
 'Dairy                                   15092
 'Dried Beans                             14256
 'Pasta & Noodles'                        11615
 'Condiments & Salad Dressings'           11254
 'Food & Beverage Gifts'                  10079
 'Breads & Bakery'                         9285
 'Meat & Seafood'           

In [178]:
new_df1.rename({'1': "category" }, inplace = True)

In [179]:
new_df1

0                       'Herbs
1                  'Beverages'
2                  'Beverages'
3                       'Herbs
4                'Snack Foods'
                  ...         
1143855     'Cooking & Baking'
1143856          'Snack Foods'
1143857            'Beverages'
1143858              'Produce'
1143859                 'Herbs
Name: 1, Length: 1143860, dtype: object

In [180]:
df.drop(['category'], axis=1, inplace=True)

In [181]:
df = pd.concat((df, new_df1), axis = 1)

In [182]:
df = df.rename(columns={1: 'category'})

In [183]:
df.sample(5)

,overall,verified,asin,reviewerName,unixReviewTime,style,image_x,userid,itemid,rating,train,description,title,brand,rank,also_view,main_cat,price,also_buy,image_y,details,reviewTime,category
478070,5.0,0,B004DIXSA4,Other Customer,1322006400,NaN,no,98847,13313,1.0,1,[Bubble Yum is the original soft and juicy bub...,"Hersheys Bubble Yum Regular, 5-Count (Pack of 18)",Bubble Yum,"38,807inGroceryGourmetFood(",NaN,Grocery,$13.65,"[B00DMJFS4Y, B001IZJPCK, B001M08XBU, B003WEHON...",yes,"\n <div class=""content"">\n\n\n\n\n\n\n<ul...",2011-11-23,'Candy & Chocolate'
187017,5.0,1,B0118FO9HK,Other Customer,1527984000,"{'Size:': ' 1 Pack', 'Flavor:': ' Caramel'}",no,126730,34602,1.0,1,[Our caramel popcorn is not just good...its AM...,Amish Good Premium Caramel Popcorn Hand Stirre...,Amish Good,"17,997inGroceryGourmetFood(",NaN,Grocery,$9.99,"[B0096DDZM2, B07F1DX6F1, B0088A3HL8, B07BLNS4B...",yes,"\n\n\n\n\n\n <div class=""disclaim"">Flavor: ...",2018-06-03,'Snack Foods'
692692,5.0,1,B0084I208A,Other Customer,1523059200,NaN,no,86811,19126,1.0,1,"[This blend has sweet, creamy caramel notes dr...","Cameron's Coffee Single Serve Pods, Flavored, ...",Cameron's Coffee,">#9,253 in Grocery & Gourmet Food (See top 100...",NaN,Grocery,$8.00,"[B00I9L306S, B00MJZ0P7W, B019FWOZNQ, B00BJ9ST6...",yes,NaN,2018-04-07,'Beverages'
226338,5.0,1,B004AFNKQ2,Other Customer,1365120000,NaN,no,61207,13023,1.0,1,[Sumatra is one of the world's more exotic cof...,Green Mountain Fair Trade Organic Sumatran Res...,Green Mountain Coffee Roasters,"31,232inGroceryGourmetFood(",NaN,Grocery,$53.90,"[B007UOMOHG, B001EO5Y52, B00I5FWX7K, B006N3I3Q...",yes,"\n <div class=""content"">\n\n\n\n\n\n\n<ul...",2013-04-05,'Beverages'
936842,NaN,1,B000FZYMJK,Other Customer,1523404800,{'Flavor:': ' Black Japonica'},no,124971,2290,NaN,0,[Committed to producing the finest quality ric...,"Lundberg Family Farms Wehani Rice, 16 Ounce (P...",Lundberg,"74,230inGroceryGourmetFood(",NaN,Grocery,$29.78,"[B000G82L62, B00DQRAP2C, B005763JMI, B006R1SQM...",yes,"\n\n\n\n\n\n <div class=""disclaim"">Flavor: ...",2018-04-11,'Dried Beans


### description

In [184]:
df.description[5]

["Personality : an all-around classic lovely and gentle. Herbal power : calms your nervous system, soothes and supports your digestion. Reason to love : one of the most widely used and well-loved herbs of the world, we adore chamomile for its mildly bittersweet, flowery taste and its ability to soothe and calm the body, mind and spirit. Our chamomile is a cut above the common chamomile you'll find in most teas. Made from only the white and yellow, tiny, daisy-like flowers (no stems and leaves here), we think of our chamomile tea as a cup of peaceful, easy feelings plain and simple. Taste fragrantly floral, honey-like and bittersweet. *The statements have not been evaluated by the food and drug administration. This product is not intended to diagnose, treat, cure or prevent any disease.",
 'About the Manufacturer:For nearly forty years, we’ve been passionate about connecting people with plants and passing along the centuries-old wisdom of how to use them. So even though our packaging ha

В данном столбце описывается состав, рецептура и пр, то, что есть на упаковке.

In [185]:
df.drop(['description'], axis=1, inplace=True)

### title

In [186]:
df.title[5]

'Traditional Medicinals Organic Chamomile Herbal Leaf Tea, 16 Tea Bags (Pack of 6)'

In [187]:
df.query('train==1').groupby('title')['overall'].agg(['count', 'mean']).sort_values(by='count').tail(10)

,count,mean
title,,
"Davidson's Tea Bulk, Anise Seed, 16-Ounce Bag",1882,4.380446
"Planters Dry Roasted Peanuts, Dry Roasted, Lightly Salted, 16 Ounce (Pack of 12)",1945,4.539846
"Planters Peanuts, Honey Roasted & Salted, 52 Ounce Canister (Pack of 2)",1951,4.585853
"Twinings of London Pure Camomile Herbal Tea Bags, 20 Count",2126,4.630292
"Twinings of London English Afternoon Black Tea Bags, 20 Count (Pack of 6)",2180,4.636697
"YumEarth Organic Lollipops, Assorted Flavors, 50 Lollipops",2300,4.744348
"KIND Bars, Caramel Almond and Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count",3182,4.534884
"Kind Bars, Madagascar Vanilla Almond, Gluten Free, Low Sugar, 1.4oz",4654,4.582725
"KIND Bars, Dark Chocolate Chili Almond, Gluten Free, 1.4 Ounce Bars, 12 Count",4733,4.578703


Оставим один их столбцов: либо этот, либо категория. В категории больше информации

In [188]:
df = df.drop(['title'], axis = 1)

### brand

In [189]:
col_info(df.brand)

Количество пропусков: 8366,
count     1135494
unique       8869
top          KIND
freq        30287
Name: brand, dtype: object,
Распределение:
KIND                              30287
Bob's Red Mill                    18855
Planters                          14214
Twinings                          12232
Green Mountain Coffee Roasters     9016
                                  ...  
Shapeworks                            2
Novus Tea                             1
Mallo Cup                             1
Appian Way                            1
cocomama                              1
Name: brand, Length: 8869, dtype: int64,


In [190]:
df.brand = df.brand.fillna('Unknown')

### rank

In [191]:
df['rank'][1]

'5,591inGroceryGourmetFood('

In [192]:
# Приведем все к str
df['rank'] = df['rank'].astype('str')
# Удалим все значения кроме числовых
df['rank'] = df['rank'].apply(lambda x: re.sub('\D', '', x))
# Заменим пропуски на высокий ранг (предположим, что чем ниже ранг, тем лучше)
df['rank'] = df['rank'].apply(lambda x: 9999999 if x == '' else x)
df['rank'] = df['rank'].astype('float')
# Остаются какие-то большие значения, заменим их на высокий ранг
df['rank'] = df['rank'].apply(lambda x: 9999999 if x > 9999999 else x)
df['rank'] = df['rank'].astype('int32')
# Посмотрим на результат
col_info(df['rank'])

Количество пропусков: 0,
count    1.143860e+06
mean     6.909507e+05
std      2.332963e+06
min      4.000000e+00
25%      1.094800e+04
50%      4.955300e+04
75%      1.822500e+05
max      9.999999e+06
Name: rank, dtype: float64,
Распределение:
9999999    66274
18         10024
2949        6228
4575        6221
72          3387
           ...  
187815         1
255650         1
478636         1
160137         1
91512          1
Name: rank, Length: 35994, dtype: int64,


### also_view

In [193]:
df['also_view'][1]

nan

Похоже это артиклы товаров аналогично столбцу asin.  Возможно для нашей системы это полезная информация. Не знаю пока что делать

In [194]:
df.sample(1)

,overall,verified,asin,reviewerName,unixReviewTime,style,image_x,userid,itemid,rating,train,brand,rank,also_view,main_cat,price,also_buy,image_y,details,reviewTime,category
895546,NaN,1,B000IZ0OB2,Other Customer,1197244800,{'Flavor:': ' Cinnamon Mocha'},no,41950,2890,NaN,0,Caffe D'Vita,219584,"[B0018VDKHU, B000H28ADA, B003NDDNQ4, B002SJUNU...",Grocery,$41.49,"[B000H28ADA, B0018VDKHU, B000H2B2A8, B000IZ24N...",no,"\n\n\n\n\n\n <div class=""disclaim"">Flavor: ...",2007-12-10,'Beverages'


In [195]:
df.columns

Index(['overall', 'verified', 'asin', 'reviewerName', 'unixReviewTime',
       'style', 'image_x', 'userid', 'itemid', 'rating', 'train', 'brand',
       'rank', 'also_view', 'main_cat', 'price', 'also_buy', 'image_y',
       'details', 'reviewTime', 'category'],
      dtype='object')

### main_cat

In [196]:
df['main_cat'][1]

'Grocery'

In [197]:
col_info(df['main_cat'])

Количество пропусков: 1299,
count     1142561
unique         17
top       Grocery
freq      1080636
Name: main_cat, dtype: object,
Распределение:
Grocery                      1080636
Health & Personal Care         45828
Amazon Home                     9769
All Beauty                      1976
Sports & Outdoors               1489
Industrial & Scientific         1226
Office Products                  854
Toys & Games                     414
Pet Supplies                     171
Tools & Home Improvement          74
Baby                              61
Arts, Crafts & Sewing             25
Home Audio & Theater              11
Camera & Photo                     9
Musical Instruments                8
Software                           5
Cell Phones & Accessories          5
Name: main_cat, dtype: int64,


Заменим пропущенные значения на неизвестно.

In [198]:
df.main_cat = df.main_cat.fillna('Unknown')

Преобразован столбец с категориями без пропусков. Этот столбец удалим

In [199]:
df = df.drop(['main_cat'], axis = 1)

### price

In [200]:
col_info(df['price'])

Количество пропусков: 348139,
count     795721
unique      4603
top       $14.99
freq       18899
Name: price, dtype: object,
Распределение:
$14.99     18899
$13.67     14471
$19.99     12637
$14.79      7583
$15.53      7308
           ...  
$109.97        3
$49.06         2
$264.87        2
$275.00        1
$6.93          1
Name: price, Length: 4603, dtype: int64,


In [201]:
df['price'][4]

nan

In [202]:
# убираем $
df.loc[~df.price.isna(), 'price'] = df.loc[~df.price.isna(), 'price'].apply(lambda x: x.replace('$', ''))

In [203]:
def med_price(row):
    """
    Вычисляет среднее между диапазоном цен
    """
    price1, trash, price2 = row['price'].split(' ')
    price = (float(price1) + float(price2)) / 2
    row['price'] = price
    return row

In [204]:
df['price'] = df['price'].apply(lambda x: 0 if x==np.nan else x)

df.loc[(df['price'].str.contains('-')) & (~df['price'].isna())] = df.loc[
    (df['price'].str.contains('-')) & (~df['price'].isna())].apply(med_price, axis=1)

In [205]:
# преобразуем и заполним пропуски
df['price'] = df.price.astype(float)
mean_price = df['price'].mean()
df['price'].fillna(mean_price, inplace=True)
df['price'] = df['price'].round(0)

In [206]:
col_info(df['price'])

Количество пропусков: 0,
count    1.143860e+06
mean     2.119466e+01
std      1.643644e+01
min      0.000000e+00
25%      1.400000e+01
50%      2.100000e+01
75%      2.200000e+01
max      9.990000e+02
Name: price, dtype: float64,
Распределение:
21.0     365647
15.0      54385
14.0      44320
10.0      34317
12.0      32379
          ...  
109.0         5
137.0         5
157.0         4
265.0         2
275.0         1
Name: price, Length: 172, dtype: int64,


### also_buy

In [207]:
df['also_buy'][4]

['B000R7XBF2',
 'B000HIFA8W',
 'B000R7RODW',
 'B07BDQGY1S',
 'B072M1NC4M',
 'B000VK61KM',
 'B000KGW2BY',
 'B00CWYM3OI',
 'B074N9Y8P9',
 'B00R0OCAMU',
 'B008GVJ9S4',
 'B0058AMYTC',
 'B001HXI0V0',
 'B00F0C0S2I',
 'B00CWYM3G6',
 'B00HKLK458',
 'B00ISYMFIK',
 'B01BMSBYVC',
 'B00BFA8BF0',
 'B079ZQNYML',
 'B0029JE7RC',
 'B077K4CVJ7',
 'B00ICT73A6',
 'B00FYR5HS4',
 'B000R7TE9Y',
 'B073S4TP58',
 'B000EQYSLC',
 'B00FQGOY6E',
 'B000F9Z2C2',
 'B0042DIZIU',
 'B011FEEBTU',
 'B002C50XBW',
 'B01MTW73ZG',
 'B0029JI9OE',
 'B00R29LF24',
 'B003ZXEBOK',
 'B0792LDHX6',
 'B000IXUK2W',
 'B00C3YAF7O',
 'B014ABUQVC',
 'B00LAQ4Z1I',
 'B00CF5F6HU',
 'B071J7HHS3',
 'B07D83162X',
 'B0785P5MP5',
 'B000IXUK8G',
 'B00NS7RWAI',
 'B072C8HBGW',
 'B004SK4KIG',
 'B00HFC2E82',
 'B071J7HQM2',
 'B0080SRA16',
 'B00O1XAOCQ',
 'B0054RPY7I',
 'B008D2X8C4',
 'B01F5EQRFC',
 'B0032ZB03G',
 'B003GADBYS',
 'B01BNIP1JW',
 'B00HU1AJU8',
 'B007PE7ANY',
 'B002ULIPL8',
 'B007NMDY38',
 'B00XA0DP86',
 'B01L6ZD7HA',
 'B00LLKWVL4',
 'B07GJTDY

Ситуация аналогична also_view

In [208]:
df.columns

Index(['overall', 'verified', 'asin', 'reviewerName', 'unixReviewTime',
       'style', 'image_x', 'userid', 'itemid', 'rating', 'train', 'brand',
       'rank', 'also_view', 'price', 'also_buy', 'image_y', 'details',
       'reviewTime', 'category'],
      dtype='object')

### image_y

In [209]:
col_info(df['image_y'])

Количество пропусков: 0,
count     1143860
unique          2
top           yes
freq      1021061
Name: image_y, dtype: object,
Распределение:
yes    1021061
no      122799
Name: image_y, dtype: int64,


### details

In [210]:
df['details'][4]

'\n\n\n\n\n\n    <div class="disclaim">Flavor: <strong>Jalapeo</strong>&nbsp;|&nbsp;Size: <strong>1.375 Ounce (Pack of 64)</strong></div>\n\n\n\n\n          <div class="content">\n\n\n\n\n\n\n<ul>\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                                                                \n\n\n    <li><b>\n    Product Dimensions: \n    </b>\n    19 x 9 x 12 inches ; 1.44 ounces\n    </li>\n\n\n\n\n\n\n\n\n<li><b>Shipping Weight:</b> 7.4 pounds</li>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<li><b>ASIN: </b>B000R7RN0G</li>\n\n\n\n\n\n\n  <li><b>UPC:</b> 642125333030 778554631147 028400444415</li>\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<li><b>Average Customer Review:</b> \n\n\n\n\n<span class="dpProductDetailB000R7RN0G">\n  <span class="a-declarative" data-action="a-popover" data-a-popover="{&quot;closeButton&quot;:&quot;false&quot;,&quot;max-width&quot;:&quot;700&quot;,&qu

Удалим столбец

In [211]:
df = df.drop(['details'], axis = 1)

# Удаление ненужных признаков

In [212]:
df.columns

Index(['overall', 'verified', 'asin', 'reviewerName', 'unixReviewTime',
       'style', 'image_x', 'userid', 'itemid', 'rating', 'train', 'brand',
       'rank', 'also_view', 'price', 'also_buy', 'image_y', 'reviewTime',
       'category'],
      dtype='object')

In [213]:
df = df.drop('overall', axis=1)

# Создание  признаков

In [214]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
df = pd.get_dummies(df, columns=[ 'category',], dummy_na=True)

In [215]:
df.sample()

,verified,asin,reviewerName,unixReviewTime,style,image_x,userid,itemid,rating,train,brand,rank,also_view,price,also_buy,image_y,reviewTime,category_ 'Alcoholic Beverages',category_ 'Beverages',category_ 'Breads & Bakery',category_ 'Breakfast Foods',category_ 'Candy & Chocolate',category_ 'Canned,category_ 'Condiments & Salad Dressings',category_ 'Cooking & Baking',category_ 'Dairy,category_ 'Deli',category_ 'Dried Beans,category_ 'Food & Beverage Gifts',category_ 'Fresh Flowers & Live Indoor Plants',category_ 'Frozen',category_ 'Herbs,category_ 'Jams,category_ 'Meat & Seafood',category_ 'Meat Substitutes',category_ 'Olives,category_ 'Pasta & Noodles',category_ 'Prepared Foods',category_ 'Produce',category_ 'Sauces,category_ 'Snack Foods',category_ 'Soups,category_nan
611803,0,B000GFYRHG,Other Customer,1436659200,{'Flavor:': ' Black Tea'},no,11888,2361,1.0,1,Bigelow Tea,2794,"[B0034KULLG, B006QAUHXS, B00BPELYL4, B000P4QX3...",15.0,"[B000GFYRJ4, B000GG5J18, B000GG5IXW, B000GG0BN...",yes,2015-07-12,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [216]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1143860 entries, 0 to 1143859
Data columns (total 43 columns):
 #   Column                                          Non-Null Count    Dtype         
---  ------                                          --------------    -----         
 0   verified                                        1143860 non-null  int64         
 1   asin                                            1143860 non-null  object        
 2   reviewerName                                    1143860 non-null  object        
 3   unixReviewTime                                  1143860 non-null  int64         
 4   style                                           592086 non-null   object        
 5   image_x                                         1143860 non-null  object        
 6   userid                                          1143860 non-null  int64         
 7   itemid                                          1143860 non-null  int64         
 8   rating                

In [217]:
df1 = df.copy() 
df1 = df.drop(['style','image_x','also_view','also_buy','unixReviewTime'], axis = 1)

In [218]:
train_data, test_data = train_test_split(df1[df1.train==1],random_state=32, shuffle=True)

In [219]:
test_data.shape

(214474, 38)

In [220]:
train_data.shape


(643421, 38)

In [221]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [222]:
NUM_THREADS = 4 #число потоков
NUM_COMPONENTS = 30 #число параметров вектора 
NUM_EPOCHS = 20 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [223]:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)

In [224]:
sklearn.metrics.roc_auc_score(test_data.rating,preds)

0.7465905888972981

In [225]:
preds = model.predict(df1[df1.train==0].userid.values,
                      df1[df1.train==0].itemid.values)

In [226]:
preds.min(), preds.max()

(-23.947285, 27.782858)

In [227]:
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()

In [228]:
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [229]:
normalized_preds.shape

(285965,)

In [230]:
submission.shape

(285965, 2)

In [231]:
submission['rating']= normalized_preds
submission.to_csv('submission_log.csv', index=False)

In [232]:
ratings_coo_all = sparse.coo_matrix((df1[df1.train==1]['rating'].astype(int),
                                 (df1[df1.train==1]['userid'],
                                  df1[df1.train==1]['itemid'])))

NUM_THREADS = 24 #число потоков
NUM_COMPONENTS = 160 #число параметров вектора 
NUM_EPOCHS = 25 #число эпох обучения
LEARNING_SCHEDULE = 'adagrad'
LOSS_FUNCTION = 'logistic'
RANDOM_STATE = 23
LR = 0.05

model = LightFM(learning_rate=LR, 
                loss=LOSS_FUNCTION,
                no_components=NUM_COMPONENTS, 
                learning_schedule=LEARNING_SCHEDULE,
                random_state = RANDOM_STATE)

model = model.fit(ratings_coo_all, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

preds = model.predict(df[df.train==0].userid.values,
                      df[df.train==0].itemid.values)
#sklearn.metrics.roc_auc_score(test.rating, preds)

In [233]:
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()

In [234]:
submission['rating']= normalized_preds

In [235]:
submission.to_csv('submission_fulltrain.csv', index=False)